In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns

In [2]:
## Files are the plot to atm temperature file once it has been modified for this notebook and the csv that has image and plot
env_logger = pd.read_csv('full_image_file_mod.csv').set_index('Image Name')
img_plot = pd.read_csv('finale_df_mod.csv')

In [3]:
del img_plot['Unnamed: 0']

In [4]:
## Renames the columns so it works with Emmanuel's code
img_plot.columns = ['image', 'plot']

In [5]:
## Adds all of the AZmet and Env data to the file that just had image name and plot (essentially merges them together)
img_plot['Date and Time'] = img_plot['azmet_atm_temp'] = img_plot['azmet_wind_velocity'] = img_plot['azmet_VPD'] = img_plot['azmet_solar_radiation'] = img_plot['relative_humidity'] = img_plot['env_temp'] = img_plot['env_wind'] = None

for i, row in img_plot.iterrows():
    meta = row['image']
    try:
        #print(env_logger.loc[meta])
        datetime = env_logger.loc[meta, 'Date and Time']
        azmet_temp = env_logger.loc[meta, 'azmet_atm_temp']
        azmet_wind_vel = env_logger.loc[meta, 'azmet_wind_velocity']
        azmet_vpd = env_logger.loc[meta, 'azmet_VPD']
        sol_rad = env_logger.loc[meta, 'azmet_solar_radiation']
        temp = env_logger.loc[meta, 'env_temp']
        win_vel = env_logger.loc[meta, 'env_wind']
        rel_hum = env_logger.loc[meta, 'relative_humidity']
        #print(sun_dir)
        
        img_plot.at[i, 'Date and Time'] = datetime
        img_plot.at[i, 'azmet_atm_temp'] = azmet_temp
        img_plot.at[i, 'azmet_wind_velocity'] = azmet_wind_vel
        img_plot.at[i, 'azmet_VPD'] = azmet_vpd
        img_plot.at[i, 'azmet_solar_radiation'] = sol_rad
        img_plot.at[i, 'env_temp'] = temp
        img_plot.at[i, 'env_wind'] = win_vel
        img_plot.at[i, 'relative_humidity'] = rel_hum
        
    except:
        pass
    

In [6]:
## Drops image and time columns so we can get stats on the values
df_agg = img_plot.drop(['image', 'Date and Time'], axis=1)#.set_index('plot')#.groupby('plot')

In [7]:
## Creates a dictionary that goes through the merged dataframe and calculates different statistical values for the data
temp_dict = {}
cnt = 0

for plot in df_agg['plot'].unique().tolist():
    try:
        cnt += 1 

        select_df = df_agg.set_index('plot').loc[plot]
        temp_median = select_df['azmet_atm_temp'].median()
        temp_mean = select_df['azmet_atm_temp'].mean()
        temp_std = select_df['azmet_atm_temp'].std()
        
        azmet_wind_vel = select_df['azmet_wind_velocity'].median()
        azmet_vpd = select_df['azmet_VPD'].median()
        sol_rad = select_df['azmet_solar_radiation'].median()
        temp = select_df['env_temp'].median()
        wind_vel = select_df['env_wind'].median()
        rel_hum = select_df['relative_humidity'].median()
        
        
        
        
        #print(temp_median)
        temp_dict[cnt] = {'plot': plot,
                          'median': temp_median,
                          'mean': temp_mean, 
                          'std_dev': temp_std, 
                          'azmet_wind_velocity': azmet_wind_vel, 
                          'azmet_VPD': azmet_vpd, 
                          'azmet_solar_radiation': sol_rad, 
                          'env_temp': temp, 
                          'env_wind': wind_vel,
                         'relative_humidity': rel_hum}
    except:
        pass

In [8]:
## Converts dictionary with stats into a dataframe and sets the plot as the index
result = pd.DataFrame.from_dict(temp_dict, orient='index').set_index('plot')

In [9]:
## Reads in the csv with the individual plant temperatures (produced by the pipeline)
plant_detections = pd.read_csv('2020-03-03_individual_temps.csv')

In [10]:
## Adds the field information and PCT values to the already existing csv that is indexed by plot
plant_detections['norm_temp'] = plant_detections['atm_temp'] = None

for i, row in plant_detections.iterrows():

    try:
        plot = row['plot'].replace('_', ' ')
        plant_temp = row['median']
        
        temp_df = result.loc[plot]
        atm_temp = temp_df['median']
        norm_temp =  atm_temp - plant_temp
        #norm_temp = plant_temp - atm_temp
        
        azmet_wind_vel = temp_df['azmet_wind_velocity']
        azmet_vpd = temp_df['azmet_VPD']
        sol_rad = temp_df['azmet_solar_radiation']
        temp = temp_df['env_temp']
        wind_vel = temp_df['env_wind']
        rel_hum = temp_df['relative_humidity']
        
        plant_detections.at[i, 'norm_temp'] = norm_temp
        plant_detections.at[i, 'atm_temp'] = atm_temp
        
        plant_detections.at[i, 'azmet_wind_velocity'] = azmet_wind_vel
        plant_detections.at[i, 'azmet_VPD'] = azmet_vpd
        plant_detections.at[i, 'azmet_solar_radiation'] = sol_rad
        plant_detections.at[i, 'env_temp'] = temp
        plant_detections.at[i, 'env_wind'] = wind_vel
        plant_detections.at[i, 'relative_humidity'] = rel_hum
        #print(atm_temp)
    except:
        pass

In [11]:
plant_detections.to_csv('plant_detections_new.csv')